# Amazon Reviews Category Classification

In this project, we are going to use some amazon reviews data, made available by [Keith Galli](https://github.com/KeithGalli/pycon2020), and NLP techniques in order to build a category classification model. This report is divided in 4 parts:

* Setting up the data
* Training the model
* Evaluating the model
* Conclusion

## Setting up the data

We start our project by loading the data and necessary packages. The data is was previously stored in a S3 bucket and it is already divided into 2 datasets, one for training and one for testing. The aws cli is already installed and setted up locally.

In [109]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score, accuracy_score
from sklearn import svm
import spacy
import json
import boto3

bucket_name = 'melo-datascience-projects'
training_path = 'nlp/amazon-reviews/training/'
testing_path = 'nlp/amazon-reviews/test/'
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)
training_files = [file.key.replace(training_path, '') for file in bucket.objects.filter(Prefix=training_path)]
training_files

['train_Automotive.json',
 'train_Beauty.json',
 'train_Books.json',
 'train_Clothing.json',
 'train_Digital_Music.json',
 'train_Electronics.json',
 'train_Grocery.json',
 'train_Patio_Lawn_Garden.json',
 'train_Pet_Supplies.json']

In [107]:
testing_files = [file.key.replace(testing_path, '') for file in bucket.objects.filter(Prefix=testing_path)]
testing_files

['test_Automotive.json',
 'test_Beauty.json',
 'test_Books.json',
 'test_Clothing.json',
 'test_Digital_Music.json',
 'test_Electronics.json',
 'test_Grocery.json',
 'test_Patio_Lawn_Garden.json',
 'test_Pet_Supplies.json']

As we can see, both the training and testing datasets are divided in several files. Each file contains reviews for a specific category, which is going to be the prediction target of our model. The titles are self-descriptive. Let us read the files.

In [112]:
train_df = pd.DataFrame()
for file in training_files:
    obj = s3.Object(bucket_name, training_path+file).get()['Body'].read()
    train_df_aux = pd.read_json(obj.decode(), lines=True)
    train_df_aux['category'] = file.strip('train_').split('.')[0]
    train_df = pd.concat([train_df_aux, train_df], ignore_index=True)

train_df.tail()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,category
22495,5,True,"04 22, 2016",A34IOVL7T9YYSF,B0002H335A,{'Size:': ' 3 Ton'},JAF,"Exactly What I expected, delivered on time. Hi...",Highly Recommend.,1461283200,NaN,NaN,Automotive
22496,5,True,"03 27, 2018",A1LTFR6UKP7N3Q,B000CITK8S,"{'Size:': ' 12V @ 750mA', 'Color:': ' Black/Gr...",Bryan Hargrave,Got this thing taking care of my lawnmower bat...,... care of my lawnmower battery and it is wor...,1522108800,NaN,NaN,Automotive
22497,5,True,"07 22, 2016",A10DPAG6XHKI25,B0006I0MVS,"{'Size:': ' 1 inch Shaft', 'Color:': ' Beige'}",buyer,Great replacement for old brittle handles. As ...,Great replacement for old brittle handles,1469145600,NaN,NaN,Automotive
22498,5,True,"10 11, 2016",A20JW9PWVAENUP,B000BUU5VS,{'Style:': ' Slide Out Lube'},michael owens,great,great,1476144000,NaN,NaN,Automotive
22499,5,True,"07 23, 2015",A30DC6PH6NJQL9,B00017YYI6,NaN,Susan,"My husband likes it, he can fill air bags goin...",Five Stars,1437609600,NaN,NaN,Automotive


In [114]:
test_df = pd.DataFrame()
for file in testing_files:
    obj = s3.Object(bucket_name, testing_path+file).get()['Body'].read()
    test_df_aux = pd.read_json(obj.decode(), lines=True)
    test_df_aux['category'] = file.strip('test_').split('.')[0]
    test_df = pd.concat([test_df_aux, test_df], ignore_index=True)

test_df.tail()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,vote,image,category
4495,5,True,"08 26, 2015",A1SRL9RS46YFNY,B0000AXS24,pedro vazquez,Good productos,Five Stars,1440547200,{'Color:': ' Black'},NaN,NaN,Automotive
4496,5,True,"08 7, 2017",AYNPB2QQS2QUF,B000CONU1K,dino77dan,Easy to install. Highly re ommended.,Good quality locks.,1502064000,NaN,NaN,NaN,Automotive
4497,5,True,"11 1, 2015",A28YJ3RXJM0R5P,B00062YCGA,Michael,Awesome price for such good oil!,Five Stars,1446336000,{'Size:': ' 5 Quart'},NaN,NaN,Automotive
4498,5,True,"08 23, 2010",AJU1HQWCM13Y8,B000CPAEJA,John Dowdell,High quality CV joint grease. I use this greas...,High quality CV joint grease,1282521600,"{'Item Package Quantity:': ' 1', 'Package Quan...",8.0,NaN,Automotive
4499,4,True,"07 5, 2014",A2G7YC89EHUP2G,B000C9WL6A,review,Well made,Four Stars,1404518400,"{'Item Package Quantity:': ' 1', 'Package Quan...",NaN,NaN,Automotive


We have 22500 rows for training and 4500 for testing. We also have several columns, but we will only be using 2: reviewText and category. Let us remove the unnecessary columns. We will also check the category proportions.

In [115]:
columns = ['reviewText', 'category']
train_df = train_df[columns]
test_df = test_df[columns]

train_df.tail()

,reviewText,category
22495,"Exactly What I expected, delivered on time. Hi...",Automotive
22496,Got this thing taking care of my lawnmower bat...,Automotive
22497,Great replacement for old brittle handles. As ...,Automotive
22498,great,Automotive
22499,"My husband likes it, he can fill air bags goin...",Automotive


In [116]:
test_df.tail()

,reviewText,category
4495,Good productos,Automotive
4496,Easy to install. Highly re ommended.,Automotive
4497,Awesome price for such good oil!,Automotive
4498,High quality CV joint grease. I use this greas...,Automotive
4499,Well made,Automotive


In [118]:
train_df.groupby('category')['reviewText'].count()

category
Automotive           2500
Beauty               2500
Books                2500
Clothing             2500
Digital_Music        2500
Electronics          2500
Grocery              2500
Patio_Lawn_Garden    2500
Pet_Supplies         2500
Name: reviewText, dtype: int64

In [121]:
test_df.groupby('category')['reviewText'].count()

category
Automotive           500
Beauty               500
Books                500
Clothing             500
Digital_Music        500
Electronics          500
Grocery              500
Patio_Lawn_Garden    500
Pet_Supplies         500
Name: reviewText, dtype: int64

The data is perfectly balanced. Let us check if there are some null or blank values.

In [122]:
train_df['reviewText'].isna().sum()

0

In [123]:
test_df['reviewText'].isna().sum()

0

In [125]:
sum(train_df['reviewText']=='')

0

In [126]:
sum(test_df['reviewText']=='')

0

All of the reviews seem to be valid. Now we are going to take some instances from the training data in order to build a validation dataset. We are going to do it in way such as that the datasets remain balanced. After that, we are ready to start the training.

In [127]:
train_df, validation_df = train_test_split(train_df, stratify=train_df['category'], test_size=0.2, random_state=42)
train_df.tail()

,reviewText,category
3678,"Recommended by a friend. It saved my sanity, a...",Patio_Lawn_Garden
19383,These are products I use regularly and am alwa...,Beauty
5876,"Would have been great, if I received chocolate...",Grocery
5543,It's Fleischmann's yeast. The best and original.,Grocery
15190,Sharpe's Fury: Richard Sharpe & the Battle of ...,Books


In [128]:
validation_df.tail()

,reviewText,category
15778,A book about winning (at business) by someone ...,Books
5848,tasty and a worthy cause,Grocery
17689,love it...wish I could find the hand lotion!,Beauty
4394,They eliminated some of the old spray patterns...,Patio_Lawn_Garden
13001,beautiful backpack.,Clothing


In [130]:
train_df.shape

(18000, 2)

In [131]:
validation_df.shape

(4500, 2)

## Training the model

We are going to use a SVM model with a linear kernel, since it is well known that it works well with NLP. The first technique that we are going to use is the bag of words. We will be starting with the unigram approach and see how it goes.

### Bag of words (unigram)

Let us start by considering only the presence of a words, instead of its counting.

In [141]:
vectorizer = CountVectorizer(binary=True)
train_x = vectorizer.fit_transform(train_df['reviewText'])
train_y = train_df['category']

clf = svm.SVC(kernel='linear', random_state=1234)
clf.fit(train_x, train_y)

SVC(kernel='linear', random_state=1234)

In [142]:
validation_x = vectorizer.transform(validation_df['reviewText'])
validation_y = validation_df['category']
pred = clf.predict(validation_x)

accuracy_score(validation_y, pred)

0.6495555555555556

We have got an accuracy of 0.65. Let us try with the words counting.

In [149]:
vectorizer = CountVectorizer()
train_x = vectorizer.fit_transform(train_df['reviewText'])
train_y = train_df['category']

clf = svm.SVC(kernel='linear', random_state=1234)
clf.fit(train_x, train_y)

SVC(kernel='linear', random_state=1234)

In [150]:
validation_x = vectorizer.transform(validation_df['reviewText'])
validation_y = validation_df['category']
pred = clf.predict(validation_x)

accuracy_score(validation_y, pred)

0.6531111111111111

The accuracy barely changed. Let us try now with bigrams.

### Bag of words (bigrams)


In [151]:
vectorizer = CountVectorizer(binary=True, ngram_range=(1,2))
train_x = vectorizer.fit_transform(train_df['reviewText'])
train_y = train_df['category']

clf = svm.SVC(kernel='linear', random_state=1234)
clf.fit(train_x, train_y)

SVC(kernel='linear', random_state=1234)

In [152]:
validation_x = vectorizer.transform(validation_df['reviewText'])
validation_y = validation_df['category']
pred = clf.predict(validation_x)

accuracy_score(validation_y, pred)

0.6131111111111112

We have got an even worse accuracy. Not let us try again with the counting, even though the results are probably going to be very similar.

In [153]:
vectorizer = CountVectorizer(ngram_range=(1,2))
train_x = vectorizer.fit_transform(train_df['reviewText'])
train_y = train_df['category']

clf = svm.SVC(kernel='linear', random_state=1234)
clf.fit(train_x, train_y)

SVC(kernel='linear', random_state=1234)

In [154]:
validation_x = vectorizer.transform(validation_df['reviewText'])
validation_y = validation_df['category']
pred = clf.predict(validation_x)

accuracy_score(validation_y, pred)

0.62

As expected, the accuracy didn't change as much. Let us try a different technique now. We will be using the word vectors approach, which is explained [here](https://towardsdatascience.com/word-vectors-and-word-meaning-90493d13af76). We are going to use a pre trained model from the spacy package.

### Word vectors

First, we need to get an english trained model. Then we are going to get the reviews representation and train our SVM model.

In [159]:
nlp = spacy.load("en_core_web_md")
#docs = [nlp(text) for text in train_x]
#train_x_word_vectors = [x.vector for x in docs]
train_x = train_df['reviewText'].apply(lambda x: nlp(x).vector)

In [169]:
len(train_x.iloc[17654])

300

In [170]:
train_y = train_df['category']

clf = svm.SVC(kernel='linear', random_state=1234)
clf.fit(list(train_x), train_y)

SVC(kernel='linear', random_state=1234)

In [171]:
validation_x = validation_df['reviewText'].apply(lambda x: nlp(x).vector)
validation_y = validation_df['category']
pred = clf.predict(list(validation_x))

accuracy_score(validation_y, pred)

0.6828888888888889

In [173]:
clf.predict([nlp('i like story').vector])

array(['Books'], dtype=object)

In [176]:
train_x.iloc[0]

array([-0.6214337 ,  0.2888258 , -3.2300901 , -0.93520683,  2.207672  ,
        0.95901906,  0.12249144,  3.2063603 , -2.0936892 ,  1.1148852 ,
        5.0563474 ,  1.7135935 , -3.9678757 ,  1.4762245 ,  1.149085  ,
       -1.3029765 ,  1.6195872 , -1.4450793 , -0.87717205, -1.3077561 ,
        0.8333802 ,  0.32594812, -1.0239401 , -1.8591763 , -1.0053483 ,
       -1.2909113 , -2.279815  , -1.0925463 , -0.7533903 ,  1.3045331 ,
        0.94079626, -1.9479262 , -0.6206791 , -0.87162095, -0.17165014,
        0.4191171 , -0.26517233,  0.7541334 ,  3.2056208 ,  1.8574672 ,
        0.05325661,  1.0791144 ,  0.18593225, -1.370542  , -0.66602176,
        1.304664  ,  0.41699854, -2.441328  , -0.9756064 ,  0.5024555 ,
        0.00898688,  0.22571865,  0.47266847, -3.9313953 , -1.4766183 ,
       -0.49231428,  0.07823862,  1.4399368 ,  0.5581441 ,  0.46420747,
        2.8159783 , -0.76833206, -0.19428353, -1.4792243 ,  0.28354523,
        1.0847745 , -2.4992309 , -4.1040955 ,  0.02688731,  2.66